In [ ]:
#!pip install --upgrade tensorflow==1.15

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNNCell, RNN, Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.compose import ColumnTransformer

In [ ]:
df = pd.read_csv('USD_INR.csv')
changes = df['Change %'].values

In [ ]:
train_ch, test_ch = train_test_split(changes, test_size = 0.2, shuffle = False, random_state = 1)

train_ch = train_ch.reshape(-1,1)
test_ch = test_ch.reshape(-1,1)

In [ ]:
#Normalize Dataset
scaler = StandardScaler().fit(train_ch)
train_ch = scaler.transform(train_ch)
test_ch = scaler.transform(test_ch)

In [ ]:
timestep = 30

Disini saya menentukan timestep sebesar 30 karena 30 rata - rata jumlah hari dalam bulan adalah 30-31 hari

In [ ]:
#mengecek ukuran dimensi training
train_ch.shape

(7757, 1)

In [ ]:
def construct_dataset(in_prices):
  features = []
  targets = []

  for i in range(timestep, in_prices.shape[0]):
    features.append(
        in_prices[i - timestep : i]
    )
    targets.append(
        in_prices[i]
    )
    features = np.array(features, 'float32')
    targets = np.array(targets, 'float32')

    return features, targets

In [ ]:
x_train, y_train = construct_dataset(train_ch)
x_test, y_test = construct_dataset(test_ch)

In [ ]:
input_placeholder = tf.placeholder(tf.float32, [None, timestep, 1])
target_placeholder = tf.placeholder(tf.float32, [None, 1])

In [ ]:
cell = SimpleRNNCell(64, activation='relu')

rnn_layer = RNN(cell, dtype=tf.float32)
output_layer = Dense(1)

In [ ]:
out_tensor = rnn_layer(input_placeholder)
out_tensor = output_layer(out_tensor)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
# mse
loss_tensor = tf.reduce_mean(0.5 * (out_tensor - target_placeholder)**2)

# optimal rmse (root mean squared error)
rmse_tensor = tf.reduce_mean(tf.sqrt(0.5 * (out_tensor - target_placeholder)**2))

In [ ]:
optimizer = tf.train.AdamOptimizer().minimize(loss_tensor)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
saver = tf.train.Saver()

In [ ]:
num_epoch = 100

In [ ]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  train_dict = {
      input_placeholder:x_train,
      target_placeholder:y_train
  }
  test_dict = {
      input_placeholder:x_test,
      target_placeholder:y_test
  }

  best_loss = float('inf')
  for epoch in range(num_epoch):
    sess.run(optimizer, feed_dict=train_dict)

    loss = sess.run(loss_tensor, feed_dict=train_dict)
    val_loss = sess.run(loss_tensor, feed_dict=test_dict)

    rmse = sess.run(rmse_tensor, feed_dict=train_dict)
    val_rmse = sess.run(rmse_tensor, feed_dict=test_dict)

    if val_loss < best_loss:
      best_loss = val_loss
      saver.save(sess, './best_model.ckpt')
      
    if epoch % 1 == 0:
      print(f'Epoch {epoch+1} - loss {loss:.4f} - rmse {rmse:.4f} - val_loss {val_loss:.4f} - val_rmse {val_rmse:.4f}')

Epoch 1 - loss 0.1634 - rmse 0.4042 - val_loss 0.0000 - val_rmse 0.0017
Epoch 2 - loss 0.1378 - rmse 0.3712 - val_loss 0.0004 - val_rmse 0.0189
Epoch 3 - loss 0.1165 - rmse 0.3414 - val_loss 0.0013 - val_rmse 0.0360
Epoch 4 - loss 0.0980 - rmse 0.3131 - val_loss 0.0026 - val_rmse 0.0507
Epoch 5 - loss 0.0812 - rmse 0.2849 - val_loss 0.0041 - val_rmse 0.0644
Epoch 6 - loss 0.0671 - rmse 0.2590 - val_loss 0.0061 - val_rmse 0.0778
Epoch 7 - loss 0.0561 - rmse 0.2368 - val_loss 0.0080 - val_rmse 0.0892
Epoch 8 - loss 0.0466 - rmse 0.2159 - val_loss 0.0098 - val_rmse 0.0988
Epoch 9 - loss 0.0379 - rmse 0.1946 - val_loss 0.0118 - val_rmse 0.1085
Epoch 10 - loss 0.0302 - rmse 0.1738 - val_loss 0.0137 - val_rmse 0.1171
Epoch 11 - loss 0.0233 - rmse 0.1528 - val_loss 0.0160 - val_rmse 0.1266
Epoch 12 - loss 0.0172 - rmse 0.1311 - val_loss 0.0189 - val_rmse 0.1373
Epoch 13 - loss 0.0121 - rmse 0.1102 - val_loss 0.0221 - val_rmse 0.1488
Epoch 14 - loss 0.0080 - rmse 0.0895 - val_loss 0.0262 - val

Saya menggunakan rmse metrics untuk mengevaluasi hasil yang didapatkan bisa dilihat bahwa value dari rmse yang dihasilkan bertambah dari 0.0289 pada epoch 1 sampai 0.1231 pada epoch 100

In [ ]:
x = changes[-30:]
x = x.reshape(-1, 1)
x = scaler.transform(x)
x = np.array([x])

In [ ]:
changes[-1]

0.0

In [ ]:
with tf.Session() as sess:
  saver.restore(sess, './best_model.ckpt')

  pred = sess.run(out_tensor, feed_dict={input_placeholder:x})
  pred = scaler.inverse_transform(pred)
  print(f'USD to IRP Next Exchange Rate: {pred}%')

INFO:tensorflow:Restoring parameters from ./best_model.ckpt
USD to IRP Next Exchange Rate: [[-0.09903964]]%


dari hasil diatas bahwa dapat dilihat prediksi exchange rate selanjutnya 